# **FORECAST CREDITOS GENERALES | Queretaro**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
import numpy as np

# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "financiamientos" in nombre and "puebla" in nombre
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_financiamientos_2019_puebla',
 'df_financiamientos_2020_puebla',
 'df_financiamientos_2021_puebla',
 'df_financiamientos_2022_puebla',
 'df_financiamientos_2023_puebla',
 'df_financiamientos_2024_puebla']

# **Forecast créditos generales (cantidad)**

### BASE

In [2]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id','año','mes','modalidad','monto']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

In [3]:
# Lista para almacenar los DataFrames procesados
dfs = []

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    # Agrupar por 'mes', y contar el número de registros únicos
    total_creditos_df = df.groupby(['mes']).size().reset_index(name='num_registros')
    # Agregar una columna con el año correspondiente
    total_creditos_df['año'] = año
    # Seleccionar las columnas necesarias
    total_creditos_df = total_creditos_df[['año', 'mes', 'num_registros']]
    
    # Añadir el DataFrame procesado a la lista
    dfs.append(total_creditos_df)

# Concatenar todos los DataFrames procesados en uno solo
total_conteo_creditos_pachuca = pd.concat(dfs, ignore_index=True)
total_conteo_creditos_pachuca

,año,mes,num_registros
0,2019,1,865
1,2019,2,1156
2,2019,3,1437
3,2019,4,1046
4,2019,5,1007
...,...,...,...
62,2024,3,935
63,2024,4,978
64,2024,5,991
65,2024,6,1041


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Preparar datos para el modelo
X = total_conteo_creditos_pachuca[['año', 'mes']]
y = total_conteo_creditos_pachuca['num_registros']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [5]:
from sklearn.metrics import r2_score
import numpy as np

# Hacer predicciones en el conjunto de prueba
y_pred = modelo.predict(X_test)
# Calcular el coeficiente de determinación (R^2)
r2 = r2_score(y_test, y_pred)
print("Coeficiente de determinación (R^2):", r2)

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
print("Error cuadrático medio (MSE):", mse)

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)
print("Error absoluto medio (MAE):", mae)
rmse = np.sqrt(mse)
print("Raíz del error cuadrático medio (RMSE):", rmse)

Coeficiente de determinación (R^2): -1.8901632521140614
Error cuadrático medio (MSE): 50370.36973280986
Error absoluto medio (MAE): 183.0447491900954
Raíz del error cuadrático medio (RMSE): 224.43344165433516


In [6]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2024, 6),
                            'mes': range(7, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)

# Redondear las predicciones a números enteros
predicciones_enteros = np.round(predicciones).astype(int)

# Agregamos las predicciones redondeadas al DataFrame de nuevas fechas
nuevas_fechas['num_registros'] = predicciones_enteros
nuevas_fechas.head()

total_conteo_pachuca = pd.concat([total_conteo_creditos_pachuca, nuevas_fechas], ignore_index=True)
total_conteo_pachuca

,año,mes,num_registros
0,2019,1,865
1,2019,2,1156
2,2019,3,1437
3,2019,4,1046
4,2019,5,1007
...,...,...,...
92,2026,8,773
93,2026,9,797
94,2026,10,822
95,2026,11,847


In [7]:
# Asegurarse de que la columna 'año' es de tipo entero
#total_conteo_pachuca['año'] = total_conteo_pachuca['año'].astype(int)

total_conteo_pachuca = total_conteo_pachuca.groupby('año')['num_registros'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
total_conteo_pachuca['modalidad'] = 'Créditos (todos)'
#total_conteo_pachuca = total_conteo_pachuca.groupby('año').agg({'num_registros': 'sum', 'modalidad': 'first'}).reset_index()

# Función para formatear números con separadores de coma y sin decimales
def format_number(x):
    return '{:,.0f}'.format(x)  # Formatear sin decimales
# Formatear la columna 'montos' con la función personalizada
total_conteo_pachuca['num_registros'] = total_conteo_pachuca['num_registros'].map(format_number)
total_conteo_pachuca = total_conteo_pachuca.iloc[1:].reset_index(drop=True)

total_conteo_pachuca

,año,num_registros,modalidad
0,2025,"9,794",Créditos (todos)
1,2026,"8,828",Créditos (todos)
2,2019,"14,503",Créditos (todos)
3,2020,"13,507",Créditos (todos)
4,2021,"14,823",Créditos (todos)
5,2022,"12,163",Créditos (todos)
6,2023,"10,837",Créditos (todos)
7,2024,"6,534",Créditos (todos)


In [8]:
# Valores conocidos
registros_2022 = 12163
registros_2023 = 10837

# Calcula la tasa de cambio porcentual de 2022 a 2023
tasa_cambio = (registros_2023 - registros_2022) / registros_2022

# Aplica la misma tasa de cambio para estimar 2024 completo
estimacion_2024 = registros_2023 * (1 + tasa_cambio)

print("Estimación de registros en 2024:", round(estimacion_2024))


Estimación de registros en 2024: 9656


In [9]:
# Remover comas y convertir la columna 'num_registros' a enteros
total_conteo_pachuca['num_registros'] = total_conteo_pachuca['num_registros'].replace({',': ''}, regex=True).astype(int)

# Ahora intenta realizar la sustitución
total_conteo_pachuca.loc[total_conteo_pachuca['año'] == "2024", 'num_registros'] = 9656

# Verifica el cambio en el DataFrame
print(total_conteo_pachuca)


    año  num_registros         modalidad
0  2025           9794  Créditos (todos)
1  2026           8828  Créditos (todos)
2  2019          14503  Créditos (todos)
3  2020          13507  Créditos (todos)
4  2021          14823  Créditos (todos)
5  2022          12163  Créditos (todos)
6  2023          10837  Créditos (todos)
7  2024           9656  Créditos (todos)


### Modalidad 1: Nueva

In [10]:
total_conteo_creditos_pachuca = pd.DataFrame()

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    # Agrupar por 'modalidad' y contar el número de registros únicos por modalidad en cada año
    conteo_creditos_año = df.groupby('modalidad').size().reset_index(name='num_registros')
    # Agregar una columna con el año correspondiente
    conteo_creditos_año['año'] = año
    
    # Concatenar el DataFrame procesado al DataFrame final
    total_conteo_creditos_pachuca = pd.concat([total_conteo_creditos_pachuca, conteo_creditos_año], ignore_index=True)

total_conteo_creditos_pachuca

,modalidad,num_registros,año
0,1,5766,2019
1,2,5386,2019
2,3,2896,2019
3,4,455,2019
4,1,5417,2020
5,2,4561,2020
6,3,2953,2020
7,4,576,2020
8,1,5650,2021
9,2,4864,2021


In [11]:
total_conteo_creditos_pachuca = pd.DataFrame()

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    
    # Agrupar por 'modalidad', 'mes', y 'año' y contar el número de registros
    conteo_creditos_año_mes = df.groupby(['modalidad', 'mes', 'año']).size().reset_index(name='num_registros')
    
    # Concatenar el DataFrame procesado al DataFrame final
    total_conteo_creditos_pachuca = pd.concat([total_conteo_creditos_pachuca, conteo_creditos_año_mes], ignore_index=True)

# Filtrar los datos por modalidad
df_modalidad_1 = total_conteo_creditos_pachuca[total_conteo_creditos_pachuca['modalidad'] == 1]
df_modalidad_3 = total_conteo_creditos_pachuca[total_conteo_creditos_pachuca['modalidad'] == 3]

# Pivotar los DataFrames para que cada columna sea el número de registros por mes
df_modalidad_1_pivot = df_modalidad_1.pivot_table(index=['año'], columns=['mes'], values='num_registros', aggfunc='sum', fill_value=0)
df_modalidad_3_pivot = df_modalidad_3.pivot_table(index=['año'], columns=['mes'], values='num_registros', aggfunc='sum', fill_value=0)

# Mostrar los DataFrames resultantes
df_modalidad_1_pivot, df_modalidad_3_pivot

(mes    1    2    3    4    5    6    7    8    9    10   11   12
 año                                                             
 2019  374  383  444  400  527  582  487  498  461  441  453  716
 2020  380  393  465  418  388  459  415  452  495  465  505  582
 2021  406  433  571  553  460  451  447  474  425  411  418  601
 2022  366  436  528  476  477  488  437  459  391  395  409  576
 2023  302  370  410  362  416  440  381  416  350  435  371  414
 2024  320  382  339  371  391  395  404    0    0    0    0    0,
 mes    1    2    3    4    5    6    7    8    9    10   11   12
 año                                                             
 2019  155  175  233  223  241  274  260  258  254  268  250  305
 2020  168  184  277  214  174  250  251  257  253  312  270  343
 2021  162  250  304  299  330  361  318  310  254  308  291  305
 2022  141  219  302  271  330  304  291  307  211  222  221  328
 2023  169  200  285  225  277  290  270  282  302  300  268  357
 2024  18

In [12]:
X = df_modalidad_1[['año', 'mes']]
y = df_modalidad_1['num_registros']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [13]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2024,6),
                            'mes': range(7, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
#fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
#                                  'mes': np.tile(range(1, 13), 3)})
#nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 


# DF con todas las combinaciones posibles de año y mes para 2025-2026
fechas_2025_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_2023, fechas_2025_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
predicciones_enteros = np.round(predicciones).astype(int) # Redondear las predicciones a números enteros
# Agregamos las predicciones redondeadas al DataFrame de nuevas fechas
nuevas_fechas['num_registros'] = predicciones_enteros

nueva_conteo_pachuca = pd.concat([df_modalidad_1, nuevas_fechas], ignore_index=True)
nueva_conteo_pachuca = nueva_conteo_pachuca.groupby('año')['num_registros'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
nueva_conteo_pachuca['modalidad'] = 'Nueva'

# Función para formatear números con separadores de coma y sin decimales
def format_number(x):
    return '{:,.0f}'.format(x)  # Formatear sin decimales

# Formatear la columna 'num_registros' con la función personalizada
nueva_conteo_pachuca['num_registros'] = nueva_conteo_pachuca['num_registros'].map(format_number)
nueva_conteo_pachuca

,año,num_registros,modalidad
0,2019,"5,766",Nueva
1,2020,"5,417",Nueva
2,2021,"5,650",Nueva
3,2022,"5,438",Nueva
4,2023,"4,667",Nueva
5,2024,"5,101",Nueva
6,2025,"4,482",Nueva
7,2026,"4,251",Nueva


In [14]:
X = df_modalidad_3[['año', 'mes']]
y = df_modalidad_3['num_registros']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [15]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2024, 6),
                            'mes': range(7, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
#fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
#                                  'mes': np.tile(range(1, 13), 3)})
#nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# DF con todas las combinaciones posibles de año y mes para 2025-2026
fechas_2025_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_2023, fechas_2025_2026]) # Concatenamos 


# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
predicciones_enteros = np.round(predicciones).astype(int) # Redondear las predicciones a números enteros
# Agregamos las predicciones redondeadas al DataFrame de nuevas fechas
nuevas_fechas['num_registros'] = predicciones_enteros

mejoramiento_conteo_pachuca = pd.concat([df_modalidad_3, nuevas_fechas], ignore_index=True)
mejoramiento_conteo_pachuca = mejoramiento_conteo_pachuca.groupby('año')['num_registros'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
mejoramiento_conteo_pachuca['modalidad'] = 'Usada'

# Función para formatear números con separadores de coma y sin decimales
def format_number(x):
    return '{:,.0f}'.format(x)  # Formatear sin decimales

# Formatear la columna 'num_registros' con la función personalizada
mejoramiento_conteo_pachuca['num_registros'] = mejoramiento_conteo_pachuca['num_registros'].map(format_number)
mejoramiento_conteo_pachuca

,año,num_registros,modalidad
0,2019,"2,896",Usada
1,2020,"2,953",Usada
2,2021,"3,492",Usada
3,2022,"3,147",Usada
4,2023,"3,225",Usada
5,2024,"3,656",Usada
6,2025,"3,440",Usada
7,2026,"3,508",Usada


In [16]:
conteo_pachuca = pd.concat([
    total_conteo_pachuca,
    nueva_conteo_pachuca,
    mejoramiento_conteo_pachuca,
])

In [17]:
# Convertir 'num_registros' a enteros y eliminar comas
conteo_pachuca['num_registros'] = conteo_pachuca['num_registros'].apply(lambda x: int(str(x).replace(',', '')))

# Convertir 'año' a enteros para asegurar el orden correcto
conteo_pachuca['año'] = conteo_pachuca['año'].astype(int)

# Ordenar los años de acuerdo con el orden específico en 'orden_anos'
orden_anos = [2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026]
conteo_pachuca = conteo_pachuca.sort_values(by='año', key=lambda x: x.map({year: i for i, year in enumerate(orden_anos)}))

# Ahora el DataFrame debería estar ordenado correctamente
print(conteo_pachuca)

    año  num_registros         modalidad
2  2019          14503  Créditos (todos)
0  2019           5766             Nueva
0  2019           2896             Usada
1  2020           5417             Nueva
1  2020           2953             Usada
3  2020          13507  Créditos (todos)
4  2021          14823  Créditos (todos)
2  2021           5650             Nueva
2  2021           3492             Usada
3  2022           3147             Usada
3  2022           5438             Nueva
5  2022          12163  Créditos (todos)
4  2023           4667             Nueva
6  2023          10837  Créditos (todos)
4  2023           3225             Usada
7  2024           9656  Créditos (todos)
5  2024           5101             Nueva
5  2024           3656             Usada
0  2025           9794  Créditos (todos)
6  2025           4482             Nueva
6  2025           3440             Usada
1  2026           8828  Créditos (todos)
7  2026           4251             Nueva
7  2026         

In [18]:
import os
import plotly.graph_objects as go
import plotly.io as pio

# Crear una figura
fig = go.Figure()

# Graficar cada modalidad
for modalidad in conteo_pachuca['modalidad'].unique():
    df_modalidad = conteo_pachuca[conteo_pachuca['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['año'], 
        y=df_modalidad['num_registros'], 
        mode='lines+markers+text',  # Mostrar líneas, puntos y texto
        name=modalidad, 
        text=df_modalidad['num_registros'],  # Añadir el texto de los valores
        textposition="top center"  # Posicionar el texto encima de los puntos
    ))

# Configurar el diseño
fig.update_layout(
    xaxis_title="Año",
    #yaxis_title="Número de Registros",
    template="plotly_white",
    margin=dict(l=10, r=10, t=10, b=10)
)
# Configurar el eje Y para mostrar los valores completos en miles (sin 'K')
fig.update_yaxes(tickformat=",")  # ',' para formato de miles sin abreviación


# Función para exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, os.path.join(carpeta, f'{nombre_archivo}.html'))

# Exportar
guardar_grafico_como_html(fig, 'g_scatt_forecast_credit_cantidad', carpeta='assets/graficas')
fig.show()


# **3.4 Forecast Monto de créditos generales. [TOTAL]**

In [19]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id', 'año', 'mes', 'modalidad', y 'monto'
    segment_df = globals()[nombre_df][['id','año','mes','modalidad','monto']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

# Lista para almacenar los DataFrames procesados
dfs = []

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    # Agrupar por 'mes', y sumar los montos
    total_creditos_df = df.groupby(['mes']).sum().reset_index()
    # Renombrar la columna 'monto' a 'montos'
    total_creditos_df.rename(columns={'monto': 'montos'}, inplace=True)
    # Agregar una columna con el año correspondiente
    total_creditos_df['año'] = año
    # Seleccionar las columnas necesarias
    total_creditos_df = total_creditos_df[['año', 'mes', 'montos']]
    
    # Añadir el DataFrame procesado a la lista
    dfs.append(total_creditos_df)

# Concatenar todos los DataFrames procesados en uno solo
total_suma_creditos_pachuca = pd.concat(dfs, ignore_index=True)
total_suma_creditos_pachuca

,año,mes,montos
0,2019,1,6.969574e+08
1,2019,2,8.036790e+08
2,2019,3,9.491480e+08
3,2019,4,8.227998e+08
4,2019,5,1.056392e+09
...,...,...,...
62,2024,3,1.069300e+09
63,2024,4,1.255097e+09
64,2024,5,1.295967e+09
65,2024,6,1.415185e+09


In [20]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np

# Preparar datos para el modelo
X = total_suma_creditos_pachuca[['año', 'mes']]
y = total_suma_creditos_pachuca['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [21]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2024, 6),
                            'mes': range(7, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
#fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
 #                                 'mes': np.tile(range(1, 13), 3)})
#nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# DF con todas las combinaciones posibles de año y mes para 2025-2026
fechas_2025_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_2023, fechas_2025_2026])  

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
# Agregamos las predicciones al DataFrame de nuevas fechas
nuevas_fechas['montos'] = predicciones
total_pachuca = pd.concat([total_suma_creditos_pachuca, nuevas_fechas], ignore_index=True)
total_todos_pachuca = total_pachuca.groupby('año')['montos'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
total_todos_pachuca['modalidad'] = 'Créditos (todos)'
# Función para formatear números con separadores de coma y dos decimales
def format_number(x):
   return '{:,.2f}'.format(x)
# Formatear la columna 'montos' con la función personalizada
total_todos_pachuca['montos'] = total_todos_pachuca['montos'].map(format_number)
#total_todos_pachuca['montos'] = total_todos_pachuca['montos'].replace({',': ''}, regex=True).astype(float).astype(int)

total_todos_pachuca = total_todos_pachuca.iloc[1:].reset_index(drop=True)

total_todos_pachuca

,año,montos,modalidad
0,2025,"15,858,055,193.72",Créditos (todos)
1,2026,"16,437,587,914.62",Créditos (todos)
2,2019,"12,151,907,325.12",Créditos (todos)
3,2020,"12,309,473,254.28",Créditos (todos)
4,2021,"14,069,053,952.03",Créditos (todos)
5,2022,"14,297,147,018.34",Créditos (todos)
6,2023,"14,225,367,186.10",Créditos (todos)
7,2024,"8,511,895,174.32",Créditos (todos)


In [22]:
# Valores conocidos
registros_2022 = 21761177351
registros_2023 = 21311155047

# Calcula la tasa de cambio porcentual de 2022 a 2023
tasa_cambio = (registros_2023 - registros_2022) / registros_2022

# Aplica la misma tasa de cambio para estimar 2024 completo
estimacion_2024 = registros_2023 * (1 + tasa_cambio)

print("Estimación de registros en 2024:", round(estimacion_2024))

Estimación de registros en 2024: 20870439228


In [23]:
# Remover comas y convertir la columna 'num_registros' a enteros
total_todos_pachuca['montos'] = total_todos_pachuca['montos'].replace({',': ''}, regex=True).astype(float).astype(int)
# Ahora intenta realizar la sustitución
total_todos_pachuca.loc[total_todos_pachuca['año'] == "2024", 'montos'] = 20870439228


# Verifica el cambio en el DataFrame
print(total_todos_pachuca)


    año       montos         modalidad
0  2025  15858055193  Créditos (todos)
1  2026  16437587914  Créditos (todos)
2  2019  12151907325  Créditos (todos)
3  2020  12309473254  Créditos (todos)
4  2021  14069053952  Créditos (todos)
5  2022  14297147018  Créditos (todos)
6  2023  14225367186  Créditos (todos)
7  2024  20870439228  Créditos (todos)


In [24]:
df_financiamientos_2024_queretaro.shape

(5435, 17)

In [25]:
total_conteo_creditos_pachuca = pd.DataFrame()

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    # Agrupar por 'modalidad' y sumar el 'monto' por modalidad en cada año
    suma_creditos_año = df.groupby('modalidad')['monto'].sum().reset_index(name='montos')
    # Agregar una columna con el año correspondiente
    suma_creditos_año['año'] = año
    # Concatenar el DataFrame procesado al DataFrame final
    total_conteo_creditos_pachuca = pd.concat([total_conteo_creditos_pachuca, suma_creditos_año], ignore_index=True)

total_conteo_creditos_pachuca

,modalidad,montos,año
0,1,7.511689e+09,2019
1,2,1.004073e+09,2019
2,3,2.931776e+09,2019
3,4,7.043703e+08,2019
4,1,7.281259e+09,2020
5,2,7.446657e+08,2020
6,3,3.268291e+09,2020
7,4,1.015258e+09,2020
8,1,7.958777e+09,2021
9,2,7.488088e+08,2021


In [26]:
total_conteo_creditos_pachuca = pd.DataFrame()

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    
    # Agrupar por 'modalidad', 'mes', y 'año' y sumar el 'monto' por modalidad en cada año
    conteo_creditos_año_mes = df.groupby(['modalidad', 'mes', 'año'])['monto'].sum().reset_index(name='montos')
    
    # Concatenar el DataFrame procesado al DataFrame final
    total_conteo_creditos_pachuca = pd.concat([total_conteo_creditos_pachuca, conteo_creditos_año_mes], ignore_index=True)

# Filtrar los datos por modalidad
df_modalidad_1 = total_conteo_creditos_pachuca[total_conteo_creditos_pachuca['modalidad'] == 1]
df_modalidad_3 = total_conteo_creditos_pachuca[total_conteo_creditos_pachuca['modalidad'] == 3]

# Pivotar los DataFrames para que cada columna sea el número de registros por mes
df_modalidad_1_pivot = df_modalidad_1.pivot_table(index=['año'], columns=['mes'], values='montos', aggfunc='sum', fill_value=0)
df_modalidad_3_pivot = df_modalidad_3.pivot_table(index=['año'], columns=['mes'], values='montos', aggfunc='sum', fill_value=0)

# Mostrar los DataFrames resultantes
df_modalidad_1_pivot, df_modalidad_3_pivot

(mes             1             2             3             4             5   \
 año                                                                          
 2019  4.825374e+08  5.555949e+08  6.719282e+08  5.716488e+08  7.232799e+08   
 2020  5.341163e+08  5.132486e+08  6.411517e+08  5.019642e+08  4.830941e+08   
 2021  5.474847e+08  5.640995e+08  8.108537e+08  8.029015e+08  6.242659e+08   
 2022  5.211219e+08  6.308728e+08  7.969821e+08  6.548718e+08  7.526484e+08   
 2023  4.959940e+08  5.595286e+08  6.942181e+08  6.479948e+08  6.793666e+08   
 2024  5.305916e+08  6.329154e+08  5.300288e+08  6.252841e+08  6.924543e+08   
 
 mes             6             7             8             9             10  \
 año                                                                          
 2019  7.750226e+08  6.154949e+08  6.634463e+08  6.102585e+08  6.069567e+08   
 2020  6.454036e+08  5.881926e+08  6.460409e+08  7.058964e+08  5.896765e+08   
 2021  6.034347e+08  6.112749e+08  6.697253e+08  6

In [27]:
modalidad1 = df_modalidad_1#[df_modalidad_1['año'] != 2024]

In [28]:
# Crear DataFrame de fechas para predicciones solo en 2025 y 2026
fechas_prediccion = pd.DataFrame({
    'año': np.repeat([2025, 2026], 12),
    'mes': np.tile(range(1, 13), 2)
})

# Crear DataFrame de fechas para predicciones de los faltantes de 2023
fechas_2023 = pd.DataFrame({
    'año': np.repeat(2024, 6),
    'mes': range(7, 13)
})

# Concatenar los DataFrames de fechas
fechas_prediccion = pd.concat([fechas_2023, fechas_prediccion], ignore_index=True)

# Entrenar el modelo solo con datos completos hasta 2023
X_train = modalidad1[['año', 'mes']]
y_train = modalidad1['montos']
modelo.fit(X_train, y_train)

# Realizar predicciones para 2023, 2025 y 2026
predicciones = modelo.predict(fechas_prediccion)
predicciones_enteros = np.round(predicciones).astype(int)

# Agregar las predicciones al DataFrame de fechas futuras
fechas_prediccion['montos'] = predicciones_enteros

# Concatenar los datos originales y las predicciones
df_con_predicciones = pd.concat([df_modalidad_1, fechas_prediccion], ignore_index=True)
df_con_predicciones['modalidad'] = 'Nueva'

# Agrupar para obtener la suma anual por modalidad
nueva_conteo_pachuca = df_con_predicciones.groupby(['año', 'modalidad'])['montos'].sum().reset_index()

# Mostrar el DataFrame resultante
print(nueva_conteo_pachuca)

    año modalidad        montos
0  2019     Nueva  7.511689e+09
1  2020     Nueva  7.281259e+09
2  2021     Nueva  7.958777e+09
3  2022     Nueva  8.556590e+09
4  2023     Nueva  7.884178e+09
5  2024     Nueva  8.828346e+09
6  2025     Nueva  8.438069e+09
7  2026     Nueva  8.595410e+09


In [29]:
X = df_modalidad_3[['año', 'mes']]
y = df_modalidad_3['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)


LinearRegression()

In [30]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
#fechas_2023 = pd.DataFrame({'año': np.repeat(2023, 5),
 #                           'mes': range(8, 13)})

fechas_2024 = pd.DataFrame({'año': np.repeat(2024, 6),
                            'mes': range(7, 13)})
# DF con todas las combinaciones posibles de año y mes para 2024-2026
#fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
#                                  'mes': np.tile(range(1, 13), 3)})
#nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# DF con todas las combinaciones posibles de año y mes para 2025-2026
fechas_2025_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_2024, fechas_2025_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
predicciones_enteros = np.round(predicciones).astype(int) # Redondear las predicciones a números enteros
# Agregamos las predicciones redondeadas al DataFrame de nuevas fechas
nuevas_fechas['montos'] = predicciones_enteros

mejoramiento_conteo_pachuca = pd.concat([df_modalidad_3, nuevas_fechas], ignore_index=True)
mejoramiento_conteo_pachuca = mejoramiento_conteo_pachuca.groupby('año')['montos'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
mejoramiento_conteo_pachuca['modalidad'] = 'Usada'
# Función para formatear números con separadores de coma y sin decimales
#def format_number(x):
 #   return '{:,.0f}'.format(x)  # Formatear sin decimales

# Formatear la columna 'num_registros' con la función personalizada
#mejoramiento_conteo_pachuca['num_registros'] = mejoramiento_conteo_pachuca['num_registros'].map(format_number)
mejoramiento_conteo_pachuca

,año,montos,modalidad
0,2019,2.931776e+09,Usada
1,2020,3.268291e+09,Usada
2,2021,4.104820e+09,Usada
3,2022,4.032495e+09,Usada
4,2023,4.845854e+09,Usada
5,2024,6.086463e+09,Usada
6,2025,5.980775e+09,Usada
7,2026,6.505796e+09,Usada


In [31]:
conteo_pachuca = pd.concat([
    total_todos_pachuca,
    nueva_conteo_pachuca,
    mejoramiento_conteo_pachuca,
])

In [32]:
conteo_pachuca

,año,montos,modalidad
0,2025,1.585806e+10,Créditos (todos)
1,2026,1.643759e+10,Créditos (todos)
2,2019,1.215191e+10,Créditos (todos)
3,2020,1.230947e+10,Créditos (todos)
4,2021,1.406905e+10,Créditos (todos)
5,2022,1.429715e+10,Créditos (todos)
6,2023,1.422537e+10,Créditos (todos)
7,2024,2.087044e+10,Créditos (todos)
0,2019,7.511689e+09,Nueva
1,2020,7.281259e+09,Nueva


In [33]:
# Convertir 'montos' a enteros y eliminar comas, manejo de NaN
conteo_pachuca['montos'] = conteo_pachuca['montos'].apply(lambda x: int(float(x)) if pd.notna(x) else x)

# Convertir 'año' a enteros para asegurar el orden correcto
conteo_pachuca['año'] = conteo_pachuca['año'].astype(int)

# Ordenar los años de acuerdo con el orden específico en 'orden_anos'
orden_anos = [2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026]
conteo_pachuca = conteo_pachuca.sort_values(by='año', key=lambda x: x.map({year: i for i, year in enumerate(orden_anos)}))

# Ahora el DataFrame debería estar ordenado correctamente
print(conteo_pachuca)

    año       montos         modalidad
2  2019  12151907325  Créditos (todos)
0  2019   7511688568             Nueva
0  2019   2931775582             Usada
1  2020   7281258869             Nueva
1  2020   3268291032             Usada
3  2020  12309473254  Créditos (todos)
4  2021  14069053952  Créditos (todos)
2  2021   7958776545             Nueva
2  2021   4104819806             Usada
3  2022   4032495116             Usada
3  2022   8556590313             Nueva
5  2022  14297147018  Créditos (todos)
4  2023   7884177802             Nueva
6  2023  14225367186  Créditos (todos)
4  2023   4845854226             Usada
7  2024  20870439228  Créditos (todos)
5  2024   8828346348             Nueva
5  2024   6086463270             Usada
0  2025  15858055193  Créditos (todos)
6  2025   8438069292             Nueva
6  2025   5980774737             Usada
1  2026  16437587914  Créditos (todos)
7  2026   8595410337             Nueva
7  2026   6505795663             Usada


In [34]:
import os
import plotly.graph_objects as go
import plotly.io as pio

# Crear una función para formatear los valores grandes en palabras en español
def formatear_numero(valor):
    if valor >= 1_000_000_000:
        return f'{valor / 1_000_000_000:.0f} mil millones'  # Mil millones
    elif valor >= 1_000_000:
        return f'{valor / 1_000_000:.0f} millones'  # Millones
    elif valor >= 1_000:
        return f'{valor / 1_000:.0f} mil'  # Mil
    else:
        return str(valor)

# Crear la figura
fig = go.Figure()

# Graficar cada modalidad
for modalidad in conteo_pachuca['modalidad'].unique():
    df_modalidad = conteo_pachuca[conteo_pachuca['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['año'], 
        y=df_modalidad['montos'], 
        mode='lines+markers+text',  # Mostrar líneas, puntos y texto
        name=modalidad, 
        text=df_modalidad['montos'].apply(lambda x: f'${x:,.0f}'),  # Añadir el texto de los valores formateados
        textposition="top center"  # Posicionar el texto encima de los puntos
    ))

# Configurar el diseño
fig.update_layout(
    xaxis_title="Año",
    yaxis_title="Monto total anual",
    template="plotly_white",
    margin=dict(l=10, r=10, t=10, b=10)
)

# Configurar el eje Y utilizando el formateo personalizado
tickvals = [1_000, 1_000_000, 1_000_000_000,5_000_000_000,10_000_000_000,15_000_000_000,20_000_000_000,25_000_000_000, 30_000_000_000]  # Valores hasta 30 mil millones
ticktext = [formatear_numero(val) for val in tickvals]  # Formatea cada valor en palabras

fig.update_yaxes(
    tickvals=tickvals, 
    ticktext=ticktext
)

# Función para exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, os.path.join(carpeta, f'{nombre_archivo}.html'))

# Exportar
guardar_grafico_como_html(fig, 'g_scatt_forecast_credit_total', carpeta='assets/graficas')
fig.show()


# **3.4 Forecast Monto de créditos generales. [PROMEDIO]**

In [2]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id', 'año', 'mes', 'modalidad', y 'monto'
    segment_df = globals()[nombre_df][['id','año','mes','modalidad','monto']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

# Lista para almacenar los DataFrames procesados
dfs = []

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    # Agrupar por 'mes', y sumar los montos
    total_creditos_df = df.groupby(['mes']).mean().reset_index()
    # Renombrar la columna 'monto' a 'montos'
    total_creditos_df.rename(columns={'monto': 'montos'}, inplace=True)
    # Agregar una columna con el año correspondiente
    total_creditos_df['año'] = año
    # Seleccionar las columnas necesarias
    total_creditos_df = total_creditos_df[['año', 'mes', 'montos']]
    
    # Añadir el DataFrame procesado a la lista
    dfs.append(total_creditos_df)

# Concatenar todos los DataFrames procesados en uno solo
total_suma_creditos = pd.concat(dfs, ignore_index=True)
total_suma_creditos

,año,mes,montos
0,2019,1,8.057311e+05
1,2019,2,6.952240e+05
2,2019,3,6.605066e+05
3,2019,4,7.866154e+05
4,2019,5,1.049049e+06
...,...,...,...
62,2024,3,1.143637e+06
63,2024,4,1.283330e+06
64,2024,5,1.307736e+06
65,2024,6,1.359448e+06


In [3]:
total_conteo_creditos = pd.DataFrame()

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    # Agrupar por 'modalidad' y sumar el 'monto' por modalidad en cada año
    suma_creditos_año = df.groupby('modalidad')['monto'].mean().reset_index(name='montos')
    # Agregar una columna con el año correspondiente
    suma_creditos_año['año'] = año
    # Concatenar el DataFrame procesado al DataFrame final
    total_conteo_creditos = pd.concat([total_conteo_creditos, suma_creditos_año], ignore_index=True)

total_conteo_creditos

,modalidad,montos,año
0,1,1.302756e+06,2019
1,2,1.864227e+05,2019
2,3,1.012353e+06,2019
3,4,1.548067e+06,2019
4,1,1.344150e+06,2020
5,2,1.632681e+05,2020
6,3,1.106770e+06,2020
7,4,1.762600e+06,2020
8,1,1.408633e+06,2021
9,2,1.539492e+05,2021


In [4]:
total_conteo_creditos = pd.DataFrame()

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    
    # Agrupar por 'modalidad', 'mes', y 'año' y sumar el 'monto' por modalidad en cada año
    conteo_creditos_año_mes = df.groupby(['modalidad', 'mes', 'año'])['monto'].mean().reset_index(name='montos')
    
    # Concatenar el DataFrame procesado al DataFrame final
    total_conteo_creditos = pd.concat([total_conteo_creditos, conteo_creditos_año_mes], ignore_index=True)

# Filtrar los datos por modalidad
df_modalidad_1 = total_conteo_creditos[total_conteo_creditos['modalidad'] == 1]
df_modalidad_3 = total_conteo_creditos[total_conteo_creditos['modalidad'] == 3]

# Pivotar los DataFrames para que cada columna sea el número de registros por mes
df_modalidad_1_pivot = df_modalidad_1.pivot_table(index=['año'], columns=['mes'], values='montos', aggfunc='sum', fill_value=0)
df_modalidad_3_pivot = df_modalidad_3.pivot_table(index=['año'], columns=['mes'], values='montos', aggfunc='sum', fill_value=0)

# Mostrar los DataFrames resultantes
df_modalidad_1_pivot, df_modalidad_3_pivot

(mes             1             2             3             4             5   \
 año                                                                          
 2019  1.290207e+06  1.450639e+06  1.513352e+06  1.429122e+06  1.372448e+06   
 2020  1.405569e+06  1.305976e+06  1.378821e+06  1.200871e+06  1.245088e+06   
 2021  1.348484e+06  1.302770e+06  1.420059e+06  1.451901e+06  1.357100e+06   
 2022  1.423830e+06  1.446956e+06  1.509436e+06  1.375781e+06  1.577879e+06   
 2023  1.642364e+06  1.512239e+06  1.693215e+06  1.790041e+06  1.633093e+06   
 2024  1.658099e+06  1.656847e+06  1.563507e+06  1.685402e+06  1.770983e+06   
 
 mes             6             7             8             9             10  \
 año                                                                          
 2019  1.331654e+06  1.263850e+06  1.332222e+06  1.323771e+06  1.376319e+06   
 2020  1.406108e+06  1.417332e+06  1.429294e+06  1.426053e+06  1.268122e+06   
 2021  1.337993e+06  1.367505e+06  1.412923e+06  1

In [5]:
modalidad1 = df_modalidad_1#[df_modalidad_1['año'] != 2024]
modalidad1['montos'] = modalidad1['montos'].replace({',': ''}, regex=True).astype(float).astype(int)


C:\Users\yoe11\AppData\Local\Temp\ipykernel_5892\3999403270.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modalidad1['montos'] = modalidad1['montos'].replace({',': ''}, regex=True).astype(float).astype(int)


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X = modalidad1[['año', 'mes']]
y = modalidad1['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [7]:
# Crear DataFrame de fechas para predicciones solo en 2025 y 2026
fechas_prediccion = pd.DataFrame({
    'año': np.repeat([2025, 2026], 12),
    'mes': np.tile(range(1, 13), 2)
})

# Crear DataFrame de fechas para predicciones de los faltantes de 2023
fechas_2023 = pd.DataFrame({
    'año': np.repeat(2024, 6),
    'mes': range(7, 13)
})

# Concatenar los DataFrames de fechas
fechas_prediccion = pd.concat([fechas_2023, fechas_prediccion], ignore_index=True)

# Entrenar el modelo solo con datos completos hasta 2023
X_train = modalidad1[['año', 'mes']]
y_train = modalidad1['montos']
modelo.fit(X_train, y_train)

# Realizar predicciones para 2023, 2025 y 2026
predicciones = modelo.predict(fechas_prediccion)
predicciones_enteros = np.round(predicciones).astype(int)

# Agregar las predicciones al DataFrame de fechas futuras
fechas_prediccion['montos'] = predicciones_enteros

# Concatenar los datos originales y las predicciones
df_con_predicciones = pd.concat([df_modalidad_1, fechas_prediccion], ignore_index=True)
df_con_predicciones['modalidad'] = 'Nueva'

# Agrupar para obtener la suma anual por modalidad
nueva_conteo = df_con_predicciones.groupby(['año', 'modalidad'])['montos'].mean().reset_index()

# Función para formatear números con separadores de coma y sin decimales
def format_number(x):
    return '{:.0f}'.format(x)  # Formatear sin decimales

# Formatear la columna 'num_registros' con la función personalizada
nueva_conteo['montos'] = nueva_conteo['montos'].map(format_number)

# Mostrar el DataFrame resultante
print(nueva_conteo)

    año modalidad   montos
0  2019     Nueva  1319633
1  2020     Nueva  1342088
2  2021     Nueva  1402796
3  2022     Nueva  1568695
4  2023     Nueva  1685066
5  2024     Nueva  1729886
6  2025     Nueva  1831944
7  2026     Nueva  1924542


In [8]:
X = df_modalidad_3[['año', 'mes']]
y = df_modalidad_3['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [9]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
#fechas_2023 = pd.DataFrame({'año': np.repeat(2023, 5),
 #                           'mes': range(8, 13)})

fechas_2024 = pd.DataFrame({'año': np.repeat(2024, 6),
                            'mes': range(7, 13)})
# DF con todas las combinaciones posibles de año y mes para 2024-2026
#fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
#                                  'mes': np.tile(range(1, 13), 3)})
#nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# DF con todas las combinaciones posibles de año y mes para 2025-2026
fechas_2025_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_2024, fechas_2025_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
predicciones_enteros = np.round(predicciones).astype(int) # Redondear las predicciones a números enteros
# Agregamos las predicciones redondeadas al DataFrame de nuevas fechas
nuevas_fechas['montos'] = predicciones_enteros

mejoramiento_conteo = pd.concat([df_modalidad_3, nuevas_fechas], ignore_index=True)
mejoramiento_conteo = mejoramiento_conteo.groupby('año')['montos'].mean().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
mejoramiento_conteo['modalidad'] = 'Usada'
# Función para formatear números con separadores de coma y sin decimales
def format_number(x):
    return '{:.0f}'.format(x)  # Formatear sin decimales

# Formatear la columna 'num_registros' con la función personalizada
mejoramiento_conteo['montos'] = mejoramiento_conteo['montos'].map(format_number)
mejoramiento_conteo

,año,montos,modalidad
0,2019,1011134,Usada
1,2020,1086521,Usada
2,2021,1172949,Usada
3,2022,1273792,Usada
4,2023,1489754,Usada
5,2024,1691964,Usada
6,2025,1781364,Usada
7,2026,1922870,Usada


In [10]:
conteo_pachuca = pd.concat([
    #total_todos,
    nueva_conteo,
    mejoramiento_conteo,
])

In [11]:
conteo_pachuca = conteo_pachuca.round(2)
conteo_pachuca

,año,modalidad,montos
0,2019,Nueva,1319633
1,2020,Nueva,1342088
2,2021,Nueva,1402796
3,2022,Nueva,1568695
4,2023,Nueva,1685066
5,2024,Nueva,1729886
6,2025,Nueva,1831944
7,2026,Nueva,1924542
0,2019,Usada,1011134
1,2020,Usada,1086521


In [12]:
# Convertir 'montos' a enteros y eliminar comas, manejo de NaN
conteo_pachuca['montos'] = conteo_pachuca['montos'].apply(lambda x: int(float(x)) if pd.notna(x) else x)

# Convertir 'año' a enteros para asegurar el orden correcto
conteo_pachuca['año'] = conteo_pachuca['año'].astype(int)

# Ordenar los años de acuerdo con el orden específico en 'orden_anos'
orden_anos = [2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026]
conteo_pachuca = conteo_pachuca.sort_values(by='año', key=lambda x: x.map({year: i for i, year in enumerate(orden_anos)}))

# Ahora el DataFrame debería estar ordenado correctamente
print(conteo_pachuca)

    año modalidad   montos
0  2019     Nueva  1319633
0  2019     Usada  1011134
1  2020     Usada  1086521
1  2020     Nueva  1342088
2  2021     Nueva  1402796
2  2021     Usada  1172949
3  2022     Usada  1273792
3  2022     Nueva  1568695
4  2023     Usada  1489754
4  2023     Nueva  1685066
5  2024     Nueva  1729886
5  2024     Usada  1691964
6  2025     Usada  1781364
6  2025     Nueva  1831944
7  2026     Nueva  1924542
7  2026     Usada  1922870


In [13]:
import os
import plotly.graph_objects as go
import plotly.io as pio

# Crear la figura
fig = go.Figure()

# Graficar cada modalidad
modalidades = conteo_pachuca['modalidad'].unique()
#colores = ['red', 'lightgreen']  # Definir los colores a usar
for i, modalidad in enumerate(modalidades):
    df_modalidad = conteo_pachuca[conteo_pachuca['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['año'], 
        y=df_modalidad['montos'], 
        mode='lines+markers+text',  # Mostrar líneas, puntos y texto
        name=modalidad, 
        text=df_modalidad['montos'].apply(lambda x: f'${x:,.0f}'),  # Añadir el texto de los valores formateados
        textposition="top center",  # Posicionar el texto encima de los puntos
        #line=dict(color=colores[i % len(colores)]),  # Asignar el color basado en el índice
        #marker=dict(color=colores[i % len(colores)]),  # Asignar el color basado en el índice
    ))

# Configurar el diseño
fig.update_layout(
    xaxis_title="Año",
    yaxis_title="Monto promedio anual",
    yaxis_tickformat=",.0f",  # Formatear el eje y con separadores de coma y sin decimales
    template="plotly_white",
    margin=dict(l=10, r=10, t=10, b=10)
)
# Función para exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, os.path.join(carpeta, f'{nombre_archivo}.html'))

# Exportar
guardar_grafico_como_html(fig, 'g_scatt_forecast_credit_promedio', carpeta='assets/graficas')
fig.show()